<a href="https://colab.research.google.com/github/shezkh/DGInternship/blob/main/week6/file_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "modin[all]"
!pip install pyyaml
!pip install kaggle

# Upload your Kaggle API key (kaggle.json) file
from google.colab import files

uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

# Move the kaggle.json file to the required directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.4/108.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 11.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 63 bytes


In [2]:
# Download the dataset from Kaggle
!kaggle datasets download -d ealtman2019/ibm-transactions-for-anti-money-laundering-aml -f LI-Medium_Trans.csv

# Unzipping the file, this file is almost 2GB after unzipping
!unzip LI-Medium_Trans.csv.zip

 97% 538M/553M [00:03<00:00, 156MB/s]
100% 553M/553M [00:04<00:00, 140MB/s]
Archive:  LI-Medium_Trans.csv.zip
  inflating: LI-Medium_Trans.csv     


In [3]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
#import pandas as pd
import dask.dataframe as dd
import datetime
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    #df = df.(columns=sorted(df.columns))
    df = df[df.columns.sort_values()]
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


In [4]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: LI-Medium_Trans
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    - timestamp
    - from_bank
    - account
    - to_bank
    - account_1
    - amount_received
    - receiving_currency
    - amount_paid
    - payment_currency
    - payment_format
    - is_laundering

Writing file.yaml


In [5]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [6]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'LI-Medium_Trans',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['timestamp',
  'from_bank',
  'account',
  'to_bank',
  'account_1',
  'amount_received',
  'receiving_currency',
  'amount_paid',
  'payment_currency',
  'payment_format',
  'is_laundering']}

In [7]:
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import yaml
import re
import os
import warnings
import time
import ray

# Disable DtypeWarnings
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

In [8]:
# Present approach of reading the file using Pandas
def read_with_pandas(file_path, delim):
    start_time = time.time()
    df = pd.read_csv(file_path, delimiter=delim, on_bad_lines='warn')
    end_time = time.time()
    return df, end_time - start_time

In [9]:
# Present approach of reading the file using Dask
def read_with_dask(file_path, delim):
    start_time = time.time()
    df = dd.read_csv(file_path, delimiter=delim, blocksize=100000000, on_bad_lines='warn')
    end_time = time.time()
    return df, end_time - start_time

In [10]:
# Present approach of reading the file using Modin
def read_with_modin(file_path, delim):
    os.environ["MODIN_MEMORY"] = "8e9"
    ray.init(object_store_memory=8e9)
    start_time = time.time()
    df = mpd.read_csv(file_path, delimiter=delim)
    end_time = time.time()
    return df, end_time - start_time

In [11]:
# other method for handling large files
# due to resource limitation I couldn't run it

'''
!pip install vaex
import vaex

def read_with_modin(file_path, delim):
    start_time = time.time()
    df = vaex.from_csv(file_path, convert=True, chunk_size=5_000_000)
    end_time = time.time()
    return df, end_time - start_time
'''

'\n!pip install vaex\nimport vaex\n\ndef read_with_modin(file_path, delim):\n    start_time = time.time()\n    df = vaex.from_csv(file_path, convert=True, chunk_size=5_000_000)\n    end_time = time.time()\n    return df, end_time - start_time\n'

In [12]:
# read the file using config file
file_type = config_data['file_type']
file_path = "./" + config_data['file_name'] + f'.{file_type}'

# Read the file using different methods and measure computational efficiency
df1, pandas_time = read_with_pandas(file_path, config_data['inbound_delimiter'])
df2, dask_time = read_with_dask(file_path, config_data['inbound_delimiter'])
#df3, modin_time = read_with_modin(file_path, config_data['inbound_delimiter'])

Inspite of several efforts, I am getting memory issues while trying to use Modin Dataframes, so i am proceeding with it. here I am going to use pandas and dask dataframe and combine their computational speed and use the best one.

In [13]:
#comparing reading time of csv and loading into a dataframe for pandas and dask
print("Pandas dataframe Execution time:",pandas_time,"\nDask dataframe execution time:",dask_time)

Pandas dataframe Execution time: 86.38079500198364 
Dask dataframe execution time: 0.03794145584106445


In [14]:
# Choose the method with the lowest execution time
if pandas_time <= dask_time:
    df = df1
else:
    df = df2

In [15]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [16]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['timestamp', 'from_bank', 'account', 'to_bank', 'account_1',
       'amount_received', 'receiving_currency', 'amount_paid',
       'payment_currency', 'payment_format', 'is_laundering'],
      dtype='object')
columns of YAML are: ['timestamp', 'from_bank', 'account', 'to_bank', 'account_1', 'amount_received', 'receiving_currency', 'amount_paid', 'payment_currency', 'payment_format', 'is_laundering']


In [17]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed


In [18]:
# Write the file in pipe-separated text file (|) in gz format
output_file_path = "/content/output_file.txt.gz"
df.to_csv(output_file_path, sep=config_data['outbound_delimiter'], compression='gzip', index=False)

# Create a summary of the file
summary = {
    'Total number of rows': len(df),
    'Total number of columns': len(df.columns),
    'File size in GB': os.path.getsize(output_file_path) / (1024 ** 3)  # Size in GB
}

In [19]:
summary

{'Total number of rows': 31251483,
 'Total number of columns': 11,
 'File size in GB': 3.814697265625e-06}